<a href="https://colab.research.google.com/github/Takkar-915/movie_review/blob/main/movie_review_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

映画レビューの内容が肯定的か否定的かの2値分類を行うモデルを作成する。
データセットは(https://ai.stanford.edu/~amaas/data/sentiment/)
に公開されているLarge Movie Review Dataset v1.0を用いた。

In [ ]:
!pip install pyprind
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


映画レビューデータセットを読み込む

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tarfile

#tarファイルに含まれているファイルをすべて取り出す。
with tarfile.open('/content/drive/MyDrive/3年前期/知的情報システム開発/データセット/aclImdb_v1.tar.gz', 'r:gz') as tar:
  tar.extractall()

reviewとsentimentのみのデータセットに編集する

In [ ]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg' :0}
#進捗状況を確認
pbar = pyprind.ProgBar(50000)
"""
aclImdbのファイル構成としてtrainフォルダ,testフォルダの下に
それぞれpos,negフォルダがある。
以下のようにパスを結合して、レヴュー文と評価のみを取り出したデータセット
を作成する
"""

df = pd.DataFrame()
for i in ('test','train'):
  for j in ('pos', 'neg'):
    path = os.path.join(basepath,i,j)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
        txt = infile.read()
      df =df.append([[txt,labels[j]]],ignore_index=True)
      pbar.update()
df.columns = ['review','sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:14


上で作成したデータフレームをシャッフル

In [ ]:
import numpy as np

np.random.seed(0)
#ランダムに並び替える
df = df.reindex(np.random.permutation(df.index))

いったんcsvファイルとして保存

In [ ]:
df.to_csv('movie_data.csv', index = False, encoding='utf-8')

cvsファイルの中身を確認

In [ ]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(5)
#df.shape

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


ここからは、正規表現を用いてHTMLマークアップ、句読点などの不要な情報を削除

In [ ]:
import re

def preprocessor(text):

  #文字列を置換
  text = re.sub('<[^>]*>', '', text)

  emozi = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)

  text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emozi).replace('-', ''))
  return text

df['review'] = df['review'].apply(preprocessor)

正規表現の結果を確認する

In [ ]:
df.head(5)

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


テキストデータを分析するために、文書をトークン化する

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

#トークン化したものから語幹を取り出す
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

あまりに一般的に使われるような単語を除外



In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english') #あまりに一般的な単語は除外している。

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


以下より、学習を開始する

まずは、訓練データとテストデータに分割

In [ ]:
x_train = df.loc[:25000,'review'].values
y_train = df.loc[:25000,'sentiment'].values
x_test = df.loc[25000:,'review'].values
y_test = df.loc[25000:,'sentiment'].values

グリッドサーチにより最適なパラメタをもとめる。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV


tfidf = TfidfVectorizer(strip_accents=None, #ascii/unicodeに含まれないアクセント文字の置き換え
                        lowercase=False,    #アルファベットを小文字に統一する設定にしない
                        preprocessor=None)

#param_gridにおける1つ目は,TfidfVectorizerを使ってTF-IDFを算出
#param_gridにおける2つ目は,引数をFalseやNoneにすることで,単に生の単語の出現頻度に基づいてモデルを訓練している。

param_grid = [{'vect__ngram_range': [(1, 1)], 
               'vect__stop_words': [stop, None],  
               'vect__tokenizer': [tokenizer, tokenizer_porter], #トークン化の手法
               'clf__penalty': ['l1', 'l2'],  #正則化。l1かl2
               'clf__C': [10.0, 100.0]}, #正則化の強さを決めるパラメータ
              
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False], #出現頻度の逆数で重み付けするかの判断
               'vect__norm':[None],     
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
#グリッドサーチを行うGridSearchクラスをインスタンス化
gs_lr_tfidf = GridSearchCV(lr_tfidf,  #チューニングを行うモデル
                           param_grid,  #パラメタ候補値を「パラメタ名, 候補値リスト」の辞書で与える
                           scoring='accuracy',  #評価手法
                           cv=3,        #3分割交差検証
                           verbose=2,   #出力ログのレベル
                           n_jobs=-1) #コア数の指定

In [ ]:
gs_lr_tfidf.fit(x_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0, 100.0], 'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "y...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'l

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7fa11f4cf5f0>} 
CV Accuracy: 0.895


In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(x_test, y_test))

Test Accuracy: 0.899
